In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import datetime
print(tf.__version__)
print(np.__version__)

2.3.0
1.18.5


In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
mnist = np.load("../mnist.npz")
x_train, y_train, x_test, y_test = mnist['x_train'], mnist['y_train'], mnist['x_test'], mnist['y_test']

x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


In [4]:
class MyModel(Model):
    def __init__(self,**kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    
    # 转换成静态图结构
    @tf.function
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# model = tf.keras.models.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28,1)),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


# @tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


# @tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

# 因为这里训练和测试时使用了两个图机制，所以在下面直接添加到tensorboard中的话会报错

In [6]:
model = MyModel()

In [7]:
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  # 定义时间戳
import os
logdir = os.path.join("logs/"+stamp)
 

summary_writer = tf.summary.create_file_writer(logdir)  # 创建文件夹 
tf.summary.trace_on(graph=True, profiler=True)  # 开启Trace，可以记录图结构和profile信息

EPOCHS = 20

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_ds:
            train_step(x_train, y_train)
            
    with summary_writer.as_default():                               # 希望使用的记录器
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)  # 还可以添加其他自定义的变量

    # for (x_test, y_test) in test_ds:
    #     test_step(x_test, y_test)
    
    # with summary_writer.as_default():                               # 希望使用的记录器
    #     tf.summary.scalar('test_loss', test_loss.result(), step=epoch)
    #     tf.summary.scalar('test_accuracy', test_accuracy.result(), step=epoch)  # 还可以添加其他自定义的变量


    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
    
with summary_writer.as_default():    
    tf.summary.trace_export(name="model_trace", step=3, profiler_outdir=None)    # 保存Trace信息到文件

Instructions for updating:
use `tf.profiler.experimental.start` instead.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.1360192596912384, Accuracy: 95.86499786376953, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 2, Loss: 0.042961593717336655, Accuracy: 98.65666961669922, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 3, Loss: 0.023203309625387192, Accuracy: 99.25833129882812, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 4, Loss: 0.014080632477998734, Accuracy: 99.5616683959961, Test Loss: 0.0, Test Accuracy: 0.0
Epoch 5, Loss: 0.01036699023097753

TypeError: expected str, bytes or os.PathLike object, not tensorflow.python.framework.ops.EagerTensor